In [ ]:
#Importing required dependencies
import os    
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
!unzip "/content/drive/MyDrive/Mask_detection/images.zip" -d"/content/drive/MyDrive/Mask_detection"  #Unzip all the images in to Mask_detection folder

Archive:  /content/drive/MyDrive/Mask_detection/images.zip
  inflating: /content/drive/MyDrive/Mask_detection/New folder/0.jpeg  
  inflating: /content/drive/MyDrive/Mask_detection/New folder/0.txt  
  inflating: /content/drive/MyDrive/Mask_detection/New folder/1.jpeg  
  inflating: /content/drive/MyDrive/Mask_detection/New folder/1.txt  
  inflating: /content/drive/MyDrive/Mask_detection/New folder/10.jpeg  
  inflating: /content/drive/MyDrive/Mask_detection/New folder/10.txt  
  inflating: /content/drive/MyDrive/Mask_detection/New folder/11.jpeg  
  inflating: /content/drive/MyDrive/Mask_detection/New folder/11.txt  
  inflating: /content/drive/MyDrive/Mask_detection/New folder/12.jpeg  
  inflating: /content/drive/MyDrive/Mask_detection/New folder/12.txt  
  inflating: /content/drive/MyDrive/Mask_detection/New folder/13.jpeg  
  inflating: /content/drive/MyDrive/Mask_detection/New folder/13.txt  
  inflating: /content/drive/MyDrive/Mask_detection/New folder/14.jpeg  
  inflating: /c

In [ ]:
%cd "/content/drive/MyDrive/Mask_detection/images"

/content/drive/MyDrive/Mask_detection/images


In [ ]:
!ls

0.jpeg	 15.jpeg  20.jpeg  26.jpeg  31.jpeg  37.jpeg  42.jpeg  48.jpeg	7.jpeg
10.jpeg  16.jpeg  21.jpeg  27.jpeg  32.jpeg  38.jpeg  43.jpeg  49.jpeg	8.jpeg
11.jpeg  17.jpeg  22.jpeg  28.jpeg  33.jpeg  39.jpeg  44.jpeg  4.jpeg	9.jpeg
12.jpeg  18.jpeg  23.jpeg  29.jpeg  34.jpeg  3.jpeg   45.jpeg  50.jpeg
13.jpeg  19.jpeg  24.jpeg  2.jpeg   35.jpeg  40.jpeg  46.jpeg  5.jpeg
14.jpeg  1.jpeg   25.jpeg  30.jpeg  36.jpeg  41.jpeg  47.jpeg  6.jpeg


In [ ]:
!git clone "https://github.com/alexeyAB/darknet" "/content/drive/MyDrive/Mask_detection/darknet"  #Clonning darknet 

Cloning into '/content/drive/MyDrive/Mask_detection/darknet'...
remote: Enumerating objects: 14997, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 14997 (delta 18), reused 25 (delta 8), pack-reused 14951
Receiving objects: 100% (14997/14997), 13.41 MiB | 5.69 MiB/s, done.
Resolving deltas: 100% (10183/10183), done.
Checking out files: 100% (2024/2024), done.


In [ ]:
%cd /content/drive/MyDrive/Mask_detection/darknet

/content/drive/MyDrive/Mask_detection/darknet


In [ ]:
os.getcwd()

'/content/drive/My Drive/Mask_detection/darknet'

In [ ]:
!ls

3rdparty		darknet_video.py       net_cam_v4.sh
build			data		       README.md
build.ps1		image_yolov3.sh        results
cfg			image_yolov4.sh        scripts
cmake			include		       src
CMakeLists.txt		json_mjpeg_streams.sh  vcpkg.json
DarknetConfig.cmake.in	LICENSE		       video_yolov3.sh
darknet_images.py	Makefile	       video_yolov4.sh
darknet.py		net_cam_v3.sh


In [ ]:
!make #Compilation process

chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1130:10: note: ...thi

In [ ]:
%cd /content/drive/MyDrive/Mask_detection  

/content/drive/MyDrive/Mask_detection


In [ ]:
!python mask_detection_folder/creating-files-data-and-name.py  

In [ ]:
!python mask_detection_folder/creating-train-and-test-txt-files.py  #Divides the images into train and test images

In [ ]:
!darknet/darknet detector train mask_detection_folder/labelled_data.data darknet/cfg/yolov3_mask.cfg mask_weights/darknet53.conv.74 -dont_show #training the model

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.961581), count: 1, class_loss = 0.000057, iou_loss = 0.009377, total_loss = 0.009433 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 30875, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.850684), count: 1, class_loss = 0.267358, iou_loss = 0.083101, total_loss = 0.350460 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 30876, rewritten_bbox = 0.000000 % 
v3 (mse

In [ ]:
#Testing part

net = cv2.dnn.readNetFromDarknet("/content/drive/MyDrive/Mask_detection/darknet/cfg/yolov3_mask.cfg",r"/content/drive/MyDrive/Mask_detection/backup/yolov3_mask_last.weights") #Reading model and configuration file

classes = ["without mask","with mask"]


image_path = ["/content/drive/MyDrive/Mask_detection/mask_detection_folder/0.jpeg","/content/drive/MyDrive/Mask_detection/mask_detection_folder/36.jpeg","/content/drive/MyDrive/Mask_detection/mask_detection_folder/50.jpeg"]
for u in image_path:
  image = cv2.imread(u)
  height , width, _ = image.shape

  blob = cv2.dnn.blobFromImage(image , 1/255 , (416,416) ,(0,0,0) , swapRB = True , crop = False) #converting an image into specific form which can be feeded to model

  net.setInput(blob)  #Input
  output_layer_names = net.getUnconnectedOutLayersNames()

  layeroutputs = net.forward(output_layer_names)

  #Extracting details of bounding boxes, confidence and classes of the image
  boxes=[]
  confidences=[]
  class_id=[]
  
  for output in layeroutputs:
      
      for detection in output:
          score = detection[5:]
          ids = np.argmax(score)
          confidence = score[ids]
          
          if confidence > 0.5: #setting confidence thresold
              center_x = int(detection[0]*width)
              center_y = int(detection[1]*height)
              w = int(detection[2]*width)
              h = int(detection[3]*height)
            
              x = int(center_x - w/2)
              y = int(center_y - h/2)
            
              boxes.append([x,y,w,h])
              confidences.append(float(confidence))
              class_id.append(ids)

  indexes = cv2.dnn.NMSBoxes(boxes,confidences ,0.5,0.4) #Picking the boxes having high confidence

  #Plotting the bounding boxes
  font = cv2.FONT_HERSHEY_PLAIN
  colors = np.random.uniform(0,255,size=(len(boxes), 3))
  if len(indexes)>0:
      for i in indexes.flatten():
          x,y,w,h = boxes[i]
          label = str(classes[class_id[i]])
          confi = str(round(confidences[i],2))
          color = colors[i]
          cv2.rectangle(image , (x,y) , (x+w,y+h) , color ,3)
          cv2.putText(image , label +" "+confi , (x,y) ,font ,3 ,(255,255,255),2)
  
  plt.figure(figsize=(5,5))
  cv2_imshow(image)
  cv2.imwrite('output.jpg',image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.